In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
searchLFP_both_sides_oversample2_LFP256_allaritf_medcondsep

# Using 4 rawnames per subj

In [ ]:
from utils_postprocess_HPC import collectBestLFP
import sys,traceback
%load_ext autoreload
%autoreload 2
#%debug

#output_per_raw= None
try:
    savefile_rawname_format='subj'
    #savefile_rawname_format = 'subj'
    #test(100)
    # n =searchLFP_both_sides_oversample2_LFP256_allartif
    rr = collectBestLFP('searchLFP_both_sides_oversample2_LFP256_allaritf', 
                        save_result = 1, savefile_rawname_format=savefile_rawname_format,
                       output_per_raw = output_per_raw);
except Exception as e:
    print(e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    
    
    stackframe = exc_info[2].tb_next.tb_frame
    local_vars_in_fun = stackframe.f_locals
    display(exc_info, stackframe,  exc_info[2].tb_next.tb_lineno)
    #display(local_vars_in_fun)

#r = collectBestLFP('test_searchLFP', save_result = 1);

In [ ]:
bld, output_per_raw = rr

In [ ]:
bld.keys()

In [ ]:
output_per_raw.keys()

In [ ]:
bld.keys()

In [ ]:
rr = bld
keys = ['S07_off,on', 'S01_off,on', 'S03_off', 'S05_off,on', 'S04_off,on', 'S02_off,on']
keys2 = [k.split('_')[0] for k in keys]; keys2
keys_rename = dict(zip(keys,keys2) )
rr2 = dict((keys_rename[key], value) for (key, value) in rr.items())
#rr2.keys()
best_LFP_dict = rr2

In [ ]:
output_per_raw = dict((keys_rename[key], value) for (key, value) in output_per_raw.items())

In [ ]:
# d = {'x':1, 'y':2, 'z':3}  # old
# d1 = {'x':'a', 'y':'b', 'z':'c'}
# dict((d1[key], value) for (key, value) in d.items())

In [ ]:
#!ls -d /p/project/icei-hbp-2020-0012/OSCBAGDIS/data_duss/sear*

# Using 2 rawnames per subj (so medcond-restr)

In [ ]:
from utils_postprocess_HPC import collectBestLFP
import sys,traceback
%load_ext autoreload
%autoreload 2
#%debug

#output_per_raw2 = None

try:
    #test(100)
    # n =searchLFP_both_sides_oversample2_LFP256_allartif
    start_time_str = "27 January 2023 00:45:00"
    rrr = collectBestLFP('searchLFP_both_sides_oversample2_LFP256_allaritf_medcondsep', 
                        save_result = 1, savefile_rawname_format='subj,medcond_glob',
                        start_time_str = start_time_str, output_per_raw = output_per_raw2);
except Exception as e:
    print(e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    
    
    stackframe = exc_info[2].tb_next.tb_frame
    local_vars_in_fun = stackframe.f_locals
    display(exc_info, stackframe,  exc_info[2].tb_next.tb_lineno)
    #display(local_vars_in_fun)

#r = collectBestLFP('test_searchLFP', save_result = 1);

# Basic analysis

In [ ]:
bld2, output_per_raw2 = rrr

from utils_postprocess_HPC import multiLevelDict2TupleList,prepTableInfo3

opr = output_per_raw2
outputs_grouped_tpll = multiLevelDict2TupleList(opr,4,3)
outputs_grouped_tpll[0][:-1]
dfunproc = pd.DataFrame(outputs_grouped_tpll, 
    columns = ['rawname','prefix','grouping','interval_set','mcf'] )
prefixes = list( dfunproc['prefix'].unique() )

perf_to_use_list = [('XGB','all_present_features', f'best_PCA-derived_features_0.99')]

to_show = []
to_show += [('allsep','merge_nothing','basic') ]
to_show += [('trem_vs_2class','merge_movements','basic')]
to_show += [('trem_vs_all','merge_all_not_trem','basic')]
to_show += [('trem_vs_quiet','merge_nothing','trem_vs_quiet') ]

# warnings.simplefilter('error')
df, table_info_per_perf_type, table_per_perf_type = \
    prepTableInfo3(opr, prefixes=prefixes, 
    perf_to_use_list=perf_to_use_list, to_show=to_show, return_df = True)

In [ ]:
import gc; gc.collect()
from utils_postprocess_HPC import extendDf

df = extendDf(df, opr)

In [ ]:
use_prefix = False
#dfsub = df[['rawname','prefix','prefix_templ', 'num','numpts','bacc','bacc_shuffled']]
dfsub = df[['rawname','grouping','interval_set',
            'prefix','prefix_templ', 'num','numpts','bacc','bacc_shuffled','sens','spec',
           'tremor_det_perf']]

prefs_to_compare = ['onlyH_subskip8','onlyH_act_subskip8','onlyH_mob_subskip8','onlyH_compl_subskip8','allb_beta_noH_subskip8','allb_gamma_noH_subskip8','allb_tremor_noH_subskip8',
        'LFPrel_noself_subskip8','modSrc_self_subskip8','modSrc_subskip8','modLFP_subskip8']

if use_prefix:
    prefs_to_compare = ['onlyH_subskip8','onlyH_act_subskip8',
                        'onlyH_mob_subskip8','onlyH_compl_subskip8']
    prefix_role = 'prefix'
else:
    pref_templs_to_compare = ['onlyH_subskip8%%','onlyH_act_subskip8%%',
                        'onlyH_mob_subskip8%%','onlyH_compl_subskip8%%',
                             'onlyH_modLFP_subskip8%%']
    prefix_role = 'prefix_templ'

dfsub2 = dfsub[ dfsub[prefix_role].isin(pref_templs_to_compare) ]

#dfsub2 = dfsub2[ (dfsub2['grouping'] == grp) & (dfsub2['interval_set'] == it) ]
grp =  dfsub2.groupby( by=[ prefix_role,'grouping','interval_set'] )

In [ ]:
prefixes = list( df['prefix'].unique() )
prefixes

In [ ]:
df.columns

In [ ]:
dfsub_ = df.query('rawname == "S05_off"')
dfsub_= dfsub_[dfsub_['prefix'].str.endswith('_disjoint')]
#[['rawname', 'grouping', 'interval_set', 'prefix','prefix_templ', 'numpts']]
grpiset = dfsub_.groupby('interval_set')
display(grpiset.size())
grprnstd = grpiset.std().reset_index()
grprnstd[[ 'interval_set', 'numpts']]

In [ ]:
df.query('subject == [S01]')

In [ ]:
#grprn = dfsub.groupby( by=['rawname', 'interval_set'] )
#dfsub3 = dfsub[dfsub['prefix'].str.startswith('onlyH_act_only11') |\
#               dfsub['prefix'].str.startswith('onlyH_act_only12')]
grprn = dfsub.groupby( by=[ 'rawname', 'grouping','interval_set'] )
grprnstd = grprn.std()
#grprnmn = grprn.mean()
display( grprn.size() )
grprnstd['numpts']

In [ ]:
bld2.keys()

In [ ]:
best_LFP_dict = bld2

# manual

## load

In [ ]:
%load_ext autoreload
%autoreload 2

import globvars as gv
import utils
import utils_tSNE as utsne
import utils_preproc as upre

import os
import sys
import mne
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
import multiprocessing as mpr
import matplotlib as mpl
import time
import gc;
import scipy.signal as sig
import pandas as pd 

import numpy as np
import utils_postprocess_HPC as postp
import pymatreader

data_dir = gv.data_dir
from os.path import join as pjoin

light_only = True

from dateutil import parser
#start_time = parser.parse("19 Aug 2021 03:05:15")
#start_time = parser.parse("14 April 2022 10:05:15")
#start_time = parser.parse("20 April 2022 10:05:15")
start_time = parser.parse("22 April 2022 17:11:15")  #2022, 4, 22, 17, 12, 27, 37491)
end_time = parser.parse("19 Aug 2031 21:21:45")
#end_time = parser.parse("19 Aug 2021 03:41:45") # this one is just to make things faster
ndaysBefore = None
#subdir = 'nointerp'
#subdir = 'nofeatsel'
#subdir = 'searchLFP'
#subdir = 'searchLFP_both_sides'
#subdir = 'searchLFP_both_sides_oversample2'
subdir = 'searchLFP_both_sides_oversample2_LFP256_allaritf'
lookup_dir = pjoin(gv.data_dir,subdir)
recent = postp.listRecent(days=ndaysBefore, lookup_dir= lookup_dir,
                          start_time=start_time, end_time=end_time)
subjs = []
print(f'Found {len(recent)} recent files')
for lf in recent:
    st = 0
    if light_only:
        if not lf.startswith('_!'):
            continue
        st = 2
    subjs += [lf[st+1:st+4]]
subjs = list(sorted(set(subjs)))
print(subjs)

import utils_postprocess_HPC as postp
#Earliest file 19 Aug 2021 03:05:15, latest file 19 Aug 2021 21:21:45
prefixes = postp.listRecentPrefixes(days = ndaysBefore, light_only=light_only, 
                                    lookup_dir= lookup_dir,
                                   start_time=start_time, end_time=end_time)
display(prefixes)

In [ ]:
from utils_postprocess_HPC import listComputedData
r = listComputedData(subdir,prefixes,start_time, end_time, use_main_LFP_chan=0)
r

In [ ]:
# check consistency directly from files (before collecting)
from utils_postprocess_HPC import checkPrefixCollectionConsistencty
ignore_missing = False
#for  grouping_to_check in ['merge_nothing', 'merge_movements']:
for  grouping_to_check in ['merge_movements']:
    #grouping_to_check= 'merge_nothing'
    it_to_check = 'basic'

    r = checkPrefixCollectionConsistencty(subdir,prefixes,start_time, end_time,
                                        grouping_to_check, it_to_check,
                                        use_main_LFP_chan=0, light_only=1,
                                        prefixes_ignore  = [], preloaded=None)
    missing, preloaded = r
    import gc; gc.collect()
    if missing is None:
        continue
    print(missing)
    if max( [len(m) for m in missing.values()] ) != 0 and not ignore_missing:
        raise ValueError('something is missing')    


In [ ]:
# here load data in memory
prefixes_to_load = prefixes
sources_type = 'parcel_aal'  # or ''
r = postp.collectPerformanceInfo3(subjs,prefixes_to_load, nraws_used='[0-9]+',   
    start_time=start_time, end_time=end_time, use_main_LFP_chan=0,
    ndays_before=None, sources_type = sources_type, printFilenames=1,
    subdir=subdir, remove_large_items = 1,
    list_only=0, allow_multi_fn_same_prefix=0,
    use_light_files = light_only, rawname_regex_full=0)
#output_per_raw,Ximp_per_raw,gis_per_raw = r
output_per_raw,_,_ = r
print('len(output_per_raw) =', len(output_per_raw))
import gc; gc.collect()

In [ ]:
# check multiple runs that we might have done in order to arrive to this dataset
import utils_postprocess as pp
from datetime import datetime
from utils_postprocess_HPC import loadRunCorresp
from utils_postprocess import printDict
#
update_outputs = False
#
tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)
CIDs,cretimes = [],[]
for tpl in tpll:
    moc = tpl[-1]
    corresp,all_info = loadRunCorresp(moc)
    runCID = dict( moc['cmd'][0] ).get('--runCID', None) 
    moc['runstrings_creation_time'] = all_info['date_created']
    moc['runCID'] = runCID
    CIDs += [runCID]
    cretimes += [all_info['date_created']]
    
    mod_time = os.stat( moc['filename_full'] ).st_mtime
    dt = datetime.fromtimestamp(mod_time)
    moc['fname_mod_time'] = dt

CIDs = list( map(int,CIDs) ) 
CIDs_sorted = list( sorted( set( map(int,CIDs) ) ) )
print('CIDs_sorted=', CIDs_sorted )
    

if len(CIDs_sorted) > 1:
    for CID in CIDs_sorted:
        print(CID, sum( np.array(CIDs) == CID) )
    CID_most_recent = list( sorted( map(int,CIDs) ) )[-1]
    print('most recent', CID_most_recent )

    desired_CIDs = [CID_most_recent]
    newtpll = []
    for tpl in tpll:
        moc = tpl[-1]
        #print(moc['runCID'])
        if int(moc['runCID']) in desired_CIDs:
            newtpll += [tpl]

    from utils_postprocess_HPC import tupleList2multiLevelDict
    d = tupleList2multiLevelDict(newtpll)
    
    # THIS IS IRREVERSIBLE WHEN RAN TWICE
    if update_outputs:
        output_per_raw_notflt = output_per_raw
        output_per_raw_flt = d
        output_per_raw = output_per_raw_flt

In [ ]:
# check conssitency after collecting and loading
from utils_postprocess_HPC import getOutputSetInfo
from utils_postprocess_HPC import checkPrefixCollectionConsistencty
rawnames_found, groupings_found, its_found, prefixes_found = getOutputSetInfo( output_per_raw )
#output_per_raw.keys()
#tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)
preloaded = output_per_raw
#preloaded = output_per_raw
for grouping_to_check in groupings_found:
    print('   ',grouping_to_check)
    #grouping_to_check= 'merge_nothing'
    it_to_check = 'basic'
    r = checkPrefixCollectionConsistencty(subdir,prefixes,start_time, end_time,  
                                          grouping_to_check, it_to_check,
                                          use_main_LFP_chan=1, light_only=1, 
                                         prefixes_ignore  = [], preloaded=preloaded)
    missing, _ = r
    print(missing)
#     if max( [len(m) for m in missing.values()] ) != 0 and not ignore_missing:
#         raise ValueError('something is missing')    

In [ ]:
#rename when needed (handling multi-dataset items, makeing them keyed by subjet only)
change_keys = False
if change_keys:
    from utils_preproc import getRawnameListStructure
    # force rename keys outside the function (doing it inside would complicate it too much)
    newkeys = []
    ordered = list(output_per_raw.items())
    vals = []
    #%debug
    for rawkey,vv in ordered:
        rawnames_cur = rawkey.split(',')
        subjs_analyzed, subjs_analyzed_glob = \
            getRawnameListStructure(rawnames_cur, ret_glob=True)
        sl = subjs_analyzed_glob['subject_list']
        mcl = list( subjs_analyzed_glob['per_medcond'].keys() )
        assert len(sl) == 1
        newkey = f'{sl[0]}'
    #     if len(mcl) == 1:
    #         newkey = f'{sl[0]}_{mcl[0]}'
    #     elif len(rawnames_cur) == 4 or (len(rawnames_cur) == 2 and sl[0] == 'S03'):
    #         newkey = f'{sl[0]}'
        newkeys += [newkey]
        vals += [vv]

    output_per_raw = dict(zip(newkeys,vals))

# import utils_postprocess as pp
# pp.printDict(output_per_raw,max_depth=1)

In [ ]:
times = []
for tpl in newtpll:
    times += [tpl[-1]['fname_mod_time']]
min(times)

## Save 

In [ ]:
# #gv.gen_subj_info[subj].get('move_side',None)
# #help(utsne.selFeatsRegex)
# chnames_LFP_contralat_to_move
# cdk
# mult_clf_output['best_LFP']['XGB']['perf_drop']

#[n for n in list(mult_clf_output.keys()) if n.find('name') >= 0]

#mult_clf_output['filename_full']

#[tpl[-1] for tpl in outputs_grouped_tpll]

#mult_clf_output['class_labels_good_for_classif']

## Collect

In [ ]:
import utils_postprocess as pp
import json
save_result = False
outputs_grouped_tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)

cpd = 'sens,spec,F1'
metrics = ['balanced_accuracy', cpd, 'sens', 'spec']
from utils_tSNE import selBestLFP

def extractPerf(p,metric_cur):
    if metric_cur == cpd:
        v = np.array( p['sens'],p['spec'] ) 
    else:
        v = np.array( p[metric_cur] )
    return (v*100).tolist()
    
best_LFP_dict = {}
for tpl in outputs_grouped_tpll:
    subj, prefix, grp, int_type, mult_clf_output = tpl
    pd0 = mult_clf_output['best_LFP']['XGB']['perf_drop']['only']
    chnames_LFP = list( pd0.keys() )
    if not chnames_LFP[0].startswith('LFP'):
        pd0 = list(pd0.values())[0].keys()
        chnames_LFP = list( pd0.keys() )
    
    cdk = ','.join([prefix,grp,int_type])
    if subj not in best_LFP_dict:
        best_LFP_dict[subj]= {}
    best_LFP_dict[subj][cdk] = {}
    
    mainmoveside_cur = gv.gen_subj_info[subj].get('move_side',None)
    movesidelet = mainmoveside_cur[0].upper()
    
    contralat_to_move_sidelet = utils.getOppositeSideStr( movesidelet )
    _,chnames_LFP_contralat_to_move = utsne.selFeatsRegex(None,chnames_LFP,[f'LFP{contralat_to_move_sidelet}'])
    
    d = {}
    for metric in metrics:    
        print(f'   metric = {metric}')
        d[metric] ={}
        d[metric + '_shuffled'] = {}
        
        #if isinstance(subj,tuple):
        #    subj = subj[0]
        #kk, mult_clf_output = tpl
        #print(kk)
        #(prefix,grp,int_type) = kk
        #for clf_type in ['LDA','XGB']:
        #chnames_LFP_controlat_to_move
        pdrop_cl,winning_chan_cl = selBestLFP(mult_clf_output, 'XGB', chnames_LFP=chnames_LFP_contralat_to_move, 
                                              metric=metric, verbose=0)
        d[metric]['best_LFP_contralat_to_move']      = winning_chan_cl
        
        pdrop,winning_chan = selBestLFP(mult_clf_output, 'XGB', chnames_LFP=chnames_LFP, metric=metric, verbose=1)
        best_LFP = winning_chan
        #pd = dict( [(chn, (100*a).tolist()) for chn,a in list( pdrop.items() )] )
        d[metric]['best_LFP']      = best_LFP
        d[metric]['perf_drop_pct'] = pdrop

        # save actual values as well
        kn = f'all_present_features'
        perf_aver = mult_clf_output['XGB_analysis_versions'][kn]['perf_dict']['perf_aver']
        d[metric][kn] = extractPerf(perf_aver,metric)
        perf_all_feats = d[metric][kn]
        
        perf_shuffled = mult_clf_output['XGB_analysis_versions'][kn]['perf_dict']['fold_type_shuffled'][-1]
        d[metric + '_shuffled'][kn ] = extractPerf(perf_shuffled,metric)
        
        for chn in chnames_LFP:
            kn = f'all_present_features_only_{chn}'
            perf_aver = mult_clf_output['XGB_analysis_versions'][kn]['perf_dict']['perf_aver']
            d[metric][kn] = extractPerf(perf_aver,metric)
            
            perf_shuffled = mult_clf_output['XGB_analysis_versions'][kn]['perf_dict']['fold_type_shuffled'][-1]
            d[metric + '_shuffled'][kn ] = extractPerf(perf_shuffled,metric)

        for chn in chnames_LFP:
            kn = f'all_present_features_but_{chn}'
            perf_aver = mult_clf_output['XGB_analysis_versions'][kn]['perf_dict']['perf_aver']
            d[metric][kn] = extractPerf(perf_aver,metric)
            
            perf_shuffled = mult_clf_output['XGB_analysis_versions'][kn]['perf_dict']['fold_type_shuffled'][-1]
            d[metric + '_shuffled'][kn ] = extractPerf(perf_shuffled,metric)
            
            dif = np.max( np.abs( np.array(perf_all_feats) - np.array(d[metric][kn] ) ) )
            if dif > 5:
                print(tpl[:-1],metric,'but',chn,dif)
            
        d['total_num_feats'] = len( mult_clf_output['featnames_for_fit'] )
        d['total_num_datapoints'] = len( mult_clf_output['class_labels_good_for_classif'] )
        d['runCID'] = mult_clf_output['runCID']
        mtime = mult_clf_output.get('mod_time',None)
        mtime= datetime.datetime.fromtimestamp( mtime )
        d['fname_mod_time'] = mtime.strftime("%d %b %Y %H:%M")

        best_LFP_dict[subj][cdk] = d
        #if subj in best_LFP_dict:
        #    
        #else:
        #    best_LFP_dict[subj] = d

        print(subj,prefix,f'true best={best_LFP}, best_cl={winning_chan_cl}, in paper={gv.gen_subj_info[subj]["lfpchan_used_in_paper"] }')        
#json.dumps(best_LFP_dict)
#gv.code_dir

In [ ]:
subdir

## save_result

In [ ]:
save_result = 1
fname_full = pjoin(gv.data_dir, subdir, f'best_LFP_info_both_sides_ext.json')
print(fname_full)
if save_result:
    #fname = pjoin(gv.data_dir, subdir, f'best_LFP_info_both_sides.json')
    with open(fname_full, 'w') as f:
        json.dump(best_LFP_dict, f)
    import subprocess as sp
    r = sp.getoutput(f'python -m json.tool {fname_full}') 
    with open(fname_full,'w') as f:
        f.write(r)
    #print(r)

## debug

In [ ]:
len(r)

In [ ]:
#r[:2000].split('\n')

# best LFP-specific analysis to DF

In [ ]:
best_LFP_info = best_LFP_dict
from utils_postprocess_HPC import sidelet2sideTempl

In [ ]:
d1.keys()

In [ ]:
import pandas as pd
permlvl = '0.9'
rows = []
ps = []
for subjlike,d0 in best_LFP_info.items():
    rnparts = subjlike.split('_')
    subj = rnparts[0]
    if len(rnparts) > 1:
        mcstr = rnparts[1]
    for ks,d1 in d0.items():
        row = {}
        (prefix,grouping,iset) = ks.split(',')
        ps += [prefix]
        d2  = d1['balanced_accuracy']
        d2s = d1['balanced_accuracy_shuffled']
        d2p = d1[f'balanced_accuracy_perm_{permlvl}']
        
        chn = d2['best_LFP']['only']
        chn_cl = d2['best_LFP_contralat_to_move']['only']
        
        p    = d2['all_present_features']
        pb   = d2[f'all_present_features_but_{chn}']
        pbcl = d2.get(f'all_present_features_but_{chn_cl}',None)    
        
        p_s    = d2s['all_present_features']
        pb_s   = d2s[f'all_present_features_but_{chn}']
        pbcl_s = d2s.get(f'all_present_features_but_{chn_cl}',None)
        
        p_p    = d2p['all_present_features'] * 100
        pb_p   = d2p.get(f'all_present_features_but_{chn}',None)
        pbcl_p = d2p.get(f'all_present_features_but_{chn_cl}',None)
        
        #d0['balanced_accuracy']['all_present_features']
        row['prefix_long'] = prefix
        row['grouping'] = grouping
        row['interval_set'] = iset
        row['subject'] = subj
        if len(rnparts) > 1:
            row['medcond'] = mcstr
        row['bacc_all'] = p
        row['bacc_best'] = pb
        row['bacc_best_contra'] = pbcl
        
        row['numpts'] = d1['total_num_datapoints']
        
        row['bacc_all_shuffled']         = p_s
        row['bacc_best_shuffled']        = pb_s
        row['bacc_best_contra_shuffled'] = pbcl_s
        
        row[f'bacc_all_perm_{permlvl}']         = p_p
        row[f'bacc_best_perm_{permlvl}']        = pb_p
        row[f'bacc_best_contra_perm_{permlvl}'] = pbcl_p

        row['runCID'] = d1['runCID']
        
        row['num'] = d1['total_num_feats']
        row['numpts'] = d1['total_num_datapoints']
        row['fname_mod_time'] = d1['fname_mod_time']
        
        parts = prefix.split('_')
        
        if parts[-1].find('disj') >= 0:
            row['disj'] = parts[-1]             
            #i = -2
        else:
            row['disj'] = 'no'
            #i = -1
            
        i,bstr = [(i,part) for (i,part) in enumerate(parts) if part.startswith('brain')][0]
        row['brain_side'] = bstr[len('brain'):]     
        #row['brain_side'] = parts[i][len('brain'):] 
        if parts[0] == 'modLFP':
            row['modality'] = parts[0]
            i0 = 1
        else:
            row['modality'] = 'LFP,msrc'
            i0 = 0
            
        row['feat'] = '_'.join( parts[i0:i])
        
        
        row['best_chn'] = chn
        row['best_chn_contra'] = chn_cl
                
        
        sidelet = row['brain_side'][0].upper()
        if sidelet == 'B':
            sidetempl = 'BB'
        else:
            sidetempl = sidelet2sideTempl( sidelet, subj )
            sidetempl = sidetempl + sidetempl
        if row['disj'] == 'disjoint':
            subskip = 8
        elif row['disj'] == 'semidisjoint':
            subskip = 4
        elif row['disj'] == 'no':
            subskip = 1
        else:
            raise ValueError(f'wrong row[\"disj\"] = {row["disj"]}')
        row['prefix_templ'] = row['modality'] + \
            f'_{row["feat"]}_' + f'subskip{subskip}' + sidetempl
        
        rows += [row]
#         break
#     break
dfLFP = pd.DataFrame(rows)

df = dfLFP.query('disj == "disjoint"').copy()
df['best_LFP_sidelet'] = df['best_chn'].apply(lambda x: x[3])
from utils_postprocess_HPC import sidelet2sideTempl
def lbd(row):
    let = row['best_LFP_sidelet']
    r = sidelet2sideTempl(let, row['subject'] )
    return r
df['best_LFP_sidetempl'] = df.apply(lbd,1)

In [ ]:
best_LFP_info['S01']['onlyH_act_brainleft_exCB,merge_nothing,trem_vs_quiet'].keys()

In [ ]:
dfLFP.columns

In [ ]:
cols = ['subject','medcond','grouping','interval_set','prefix_templ','best_LFP_sidetempl',
        'numpts','bacc_all', 'bacc_all_shuffled', 
    f'bacc_all_perm_{permlvl}',
    'bacc_best', f'bacc_best_perm_{permlvl}', 'bacc_best_shuffled',
    'bacc_best_contra','bacc_best_contra_shuffled','feat']

In [ ]:
#df['prefix_templ'].str.contains('onlyH_act')

In [ ]:
import gc; gc.collect()

In [ ]:
## Ipsilateral or contralateral side gives best LFP?
# let's test on LFP only data, only H_act
dfc = df[df['prefix_templ'].str.endswith('BB') & \
         df['prefix_templ'].str.contains('modLFP_onlyH_act')].query(
    'grouping == "merge_nothing" and interval_set == "basic"')
dfc = dfc.sort_values(['subject','medcond'])
dfc[['subject','medcond','grouping','interval_set','prefix_templ','best_LFP_sidetempl','bacc_all']]

In [ ]:

df[cols]

In [ ]:
1030 % 256

In [ ]:
df.query('feat == ""')['prefix_long']

In [ ]:
df['modality'].unique()

In [ ]:
parts[i]

In [ ]:
row

In [ ]:
row['brain_side']

In [ ]:
dfLFP

In [ ]:
# select disjoint and restrucutre the table
# this cell needs df_all
dfLFPd = dfLFP[(dfLFP['disj'] == 'disjoint') & (dfLFP['grouping'] == 'merge_movements') \
              & (dfLFP['interval_set'] == 'basic')]

grp = dfLFPd.groupby(['prefix_templ','grouping','interval_set'])
display(grp.size())
dfLFPdm = grp.mean()
dfLFPdstd = grp.std()
display(dfLFPdm,dfLFPdstd)

In [ ]:
dfLFPdm = dfLFPdm.reset_index()
dfLFPdstd = dfLFPdstd.reset_index()

finals = []

prefsd = {'prefix':{}, 'ending':{}, 'prefix_short':{}}
prefsd['ending']['contra'] = '_subskip8%%'
prefsd['ending']['ipsi'] = '_subskip8^^'
for s,end in prefsd['ending'].items():
    prefsd['prefix'][s] = [f'modLFP_onlyH{end}',f'modLFP_onlyH_act{end}']
    prefsd['prefix_short'][s] = [f'onlyH_act{end}', f'onlyH{end}']    

    # now I want to make two tables each taking two rows
    dfLFPdm_sub = dfLFPdm[dfLFPdm['prefix_templ'].isin(prefsd['prefix'][s])]
    dfLFPdm_sub_allLFP  = dfLFPdm_sub[['prefix_templ','bacc_all','bacc_all_shuffled']]
    dfLFPdm_sub_bestLFP = dfLFPdm_sub[['prefix_templ','bacc_best','bacc_best_shuffled']]

    dfLFPdm_sub_allLFP['prefix_templ'] = dfLFPdm_sub_allLFP['prefix_templ'].str.\
        replace('_subskip','_allLFP_subskip')

    dfLFPdm_sub_allLFP = dfLFPdm_sub_allLFP.rename(columns=\
                        {'bacc_all':'bacc', 'bacc_all_shuffled':'bacc_shuffled'})
    #dfLFPdm_sub_allLFP.drop( ['bacc_best','bacc_best_shuffled'],1)

    dfLFPdm_sub_bestLFP = dfLFPdm_sub_bestLFP.rename(columns=\
                        {'bacc_best':'bacc', 'bacc_best_shuffled':'bacc_shuffled'})
    #dfLFPdm_sub_bestLFP.drop( ['bacc_all','bacc_all_shuffled'], 1)

    dfLFPdm_sub_allLFP = dfLFPdm_sub_allLFP.reset_index()
    dfLFPdm_sub_bestLFP = dfLFPdm_sub_bestLFP.reset_index()

    dfc = pd.concat([dfLFPdm_sub_allLFP,dfLFPdm_sub_bestLFP])
    #dfc = dfc.drop('level_0',1)
    dfc = dfc.drop('index',1)
    #dfc
    #--------------------------------

    df_allH = df_all[df_all['prefix_templ'].isin( prefsd['prefix_short'][s]) ]\
        [['prefix_templ','bacc', 'bacc_shuffled']]
    df_allmod = df_allH.groupby('prefix_templ').mean().reset_index()
    df_allmod['LFP channels'] = 'best'
    
    def lbd(row): 
        if row['prefix_templ'].find('allLFP') < 0:
            return 'all' 
        else:
            return 'best'
    #dfc2['LFP channels'] = None
    dfc['LFP channels'] = dfc.apply(lbd,1)
    
    dfc2 = pd.concat([dfc,df_allmod]).reset_index().drop('index',1)
    dfc2['prefix_templ'] = dfc2['prefix_templ'].str.replace(end,'', regex=0)
    dfc2['modality'] = dfc2['prefix_templ'].apply(lambda x: 'LFP + cortex' \
                                                  if x.find('modLFP') < 0 else 'LFP')
    
    dfc2['prefix_templ'] = dfc2['prefix_templ'].str.replace('modLFP_','')
    dfc2['prefix_templ'] = dfc2['prefix_templ'].str.replace('_allLFP','')

    dfc2['prefix_templ'] = dfc2['prefix_templ'].replace({'onlyH_act':'Hjorth activity',
                                                        'onlyH':'all Hjorth parameters'})

    dfc2['bacc'] = dfc2['bacc'].apply(lambda x : f'{x:.0f}%')
    dfc2['bacc_shuffled'] = dfc2['bacc_shuffled'].apply(lambda x : f'{x:.0f}%')


    rename_dict = {}
    rename_dict['prefix_templ'] = 'Feature set'
    rename_dict['modality'] = 'Data type'
    #rename_dict['interval_set'] = 'behav. states used'
    rename_dict['bacc'] = 'Performance'
    rename_dict['bacc_shuffled'] = 'Performance shuffled'
    #rename_dict['bacc_std'] = 'performance std'
    dfc2.rename(columns=rename_dict, inplace = 1)
    
    dfc2['Brain side'] = s

    dfc2r = dfc2[['Data type', 'LFP channels', 'Brain side', 'Feature set', 'Performance', 'Performance shuffled']]
    display(dfc2r)
    
    finals += [dfc2r]

# old

In [ ]:
best_LFP_dict[subj].keys()

In [ ]:
best = best_LFP_dict[subj]['modLFP_onlyH_act_brainright_disjoint,merge_nothing,basic'][metric]['best_LFP']
# 53 KeyError: 'modLFP_onlyH_act_brainright_disjoint,merge_nothing,basic'


In [ ]:
import datetime
dt = datetime.datetime.now()
dt.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
subdir

In [ ]:
from utils_postprocess import getBestLFP_clToMove
getBestLFP_clToMove(best_LFP_dict,subj)

In [ ]:
# I want to have easy to use cl to move best LFP
for subj,vals in best_LFP_dict.items():
    ks = list( vals.keys() )

In [ ]:
'_postproc','modLFP_onlyH_act_*_disjoint,merge_movements,basic'

brainright_exCB

best_LFP_dict[subj]['modLFP_onlyH_act_brainright_exCB_disjoint,merge_movements,basic']['balanced_accuracy']['best_LFP']

In [ ]:
fname = pjoin(gv.data_dir, 'best_LFP_info.json')
with open(fname, 'r') as f:
    jl = json.load(f)

In [ ]:
fname

In [ ]:
jl['S01']['modLFP,merge_movements,basic']

In [ ]:
jl['S02']['modLFP,merge_movements,basic']

In [ ]:
for kk in jl['S01'].keys():
    print(kk.split(',')[0])

In [ ]:
all_thrs = mult_clf_output['feat_variance_q_thr']
#thr0,thr1,thr2='0.6','0.75','0.9'
thr0,thr1,thr2='0.87','0.92','0.99'

all_LDA =  []
all_XGB = ['all_present_features', 
           'best_LFP']


perf_to_use_list = []
if len(all_LDA):
    for v in all_LDA[1:]:
        perf_to_use_list += [('LDA',all_LDA[0],v)]
for v in all_XGB[1:]:
    perf_to_use_list += [('XGB',all_XGB[0],v)]

###########################################
display(perf_to_use_list)

to_show = [('allsep','merge_nothing','basic'), ('trem_vs_all','merge_all_not_trem','basic'),
        ('trem_vs_2class','merge_movements','basic')]
to_show = [('trem_vs_2class','merge_movements','basic')]


# warnings.simplefilter('error')
# table_info_per_perf_type, table_per_perf_type = \
#     postp.prepTableInfo2(output_per_raw, prefixes=prefixes, 
#     perf_to_use_list=perf_to_use_list)

#%debug
# import warnings
# with warnings.catch_warnings():
#warnings.simplefilter('error')
table_info_per_perf_type, table_per_perf_type = \
    postp.prepTableInfo3(output_per_raw, prefixes=prefixes, 
    perf_to_use_list=perf_to_use_list, to_show=to_show)

In [ ]:
json.dumps(best_LFP_dict)

In [ ]:
#perf_tuple = perf_to_use_list[0]
#%debug
#for perf_tuple in perf_to_use_list[:1]:
for perf_tuple in perf_to_use_list:
    print(perf_tuple)
    postp.plotTableInfos2(table_info_per_perf_type, perf_tuple=perf_tuple, 
                          output_subdir=subdir) 
    plt.close()
    import gc;gc.collect()
#postp.plotTableInfos2(table_info_per_perf_type, perf_kind='LDA', keys = None, output_subdir=''): 

In [ ]:
help(plt.suptitle)

In [ ]:
rncur = 'S01_off_hold'
sind_str,mc,tk  = utils.getParamsFromRawname(rncur)
sources_type='parcel_aal'
src_file_grouping_ind = 10
src_rec_info_fn = '{}_{}_grp{}_src_rec_info'.format(rncur,
                                                    sources_type,src_file_grouping_ind)
src_rec_info_fn_full = os.path.join(gv.data_dir, src_rec_info_fn + '.npz')
rec_info = np.load(src_rec_info_fn_full, allow_pickle=True)


print( list(rec_info.keys()) )

labels_dict = rec_info['label_groups_dict'][()]

In [ ]:
import utils_postprocess as pp

prefixes_to_use = ['cross_freqmod_beta,gamma:HFO']
prefixes_to_use = ['LFPrel_noself']
#for prefix_cur in list(sorted( set(prefixes) - set(['all']) )):
#for prefix_cur in ['cross_freqmod_beta,gamma:HFO']:
#for prefix_cur in [ 'LFPrel_noself']:
for prefix_cur in [ 'onlyH']:
    prefixes_to_use = [prefix_cur]

    outputs_grouped = pp.groupOutputs(output_per_raw, prefixes_to_use,['merge_movements'],['basic'])
    #pp.printDict(outputs_grouped,1,print_leaves=1)


    use_light_file = 1
    perf_thr = 0.7

    rname_crop = slice(0,3)

    #(prefix,grp,int_type), output = u
    print(f'Starting plotting for {len(outputs_grouped)} grouped outputs')
    info_cur = {}
    #sens,spec = res[rn].get(pref, (np.nan, np.nan))
    (prefix,grp,int_type), mult_clf_output = list(outputs_grouped.values())[0]

    output_subdir = ''
    rnstr = ';'.join( list( outputs_grouped.values() )[0][0] )
    out_name_plot = f'{",".join((prefix,grp,int_type) ) }_feat_signif'        
    #chnames_LFP = ['LFPR01', 'LFPR12', 'LFPR23']

    str_to_put_ = ''
    #     pvec = mult_clf_output['XGB_analysis_versions']['all_present_features']['perf_dict']['perf_aver']
    #     sens,spec,F1 = pvec
    #     str_to_put_ =  '{:.0f}%,{:.0f}%,{:.0f}%'.format(100*pvec[0],100*pvec[1],100*pvec[2])
    #     if min(sens,spec) < perf_thr:
    #         print('  Skipping due to low perf ',str_to_put_)
    #         continue
    #hh = int( 20 * len(featnames) / 270  ), 
    #%debug

    #outputs_grouped   # 
    #(prefix,grp,int_type), mult_clf_output = output_list[0]
    #%debug
    from matplotlib.backends.backend_pdf import PdfPages
    try:
        pdf= PdfPages(pjoin(gv.dir_fig, subdir, out_name_plot + '.pdf' ))    
        postp.plotFeatSignifSHAP_list(pdf=None,
                                 outputs_grouped=outputs_grouped, fshs='XGB_Shapley',                                  
                                 figname_prefix=prefix, roi_labels=labels_dict['all_raw'],
                                 body_side='left', chnames_LFP=chnames_LFP, 
                                 hh = 12, 
                                 tickfontsize = 10, markersize=8,
                                 suptitle = str_to_put_, use_best_LFP=True,
                                 suptitle_fontsize=20, show_bias=0, show_max = False, 
                                 show_std = False, average_over_subjects = True,
                                 merge_Hjorth = prefix_cur != 'onlyH',alpha=0.4);                  

        pdf.savefig()
        plt.close()        

        #plt.close('all')
    except KeyError as e:
        print(e)
        continue
    
    cax,clrb = postp.plotConfmats(outputs_grouped, best_LFP=1)
    kind = ';'.join( list( outputs_grouped.values() )[0][0] )
    figname = ','.join( [k[0] for k in outputs_grouped.keys()] ) + f'_confmats_{kind}.pdf'
    kind = ';  '.join( list( outputs_grouped.values() )[0][0] )
    plt.suptitle(kind)
    #plt.savefig( pjoin(gv.dir_fig,subdir,figname) )
    
    pdf.savefig()
    plt.close()
    
    pdf.close()

#             break
#         break
#     break

In [ ]:
gc.collect()

In [ ]:
a = np.array([np.eye(3),np.eye(3)])
print( a.shape )
ais = np.where(a)
a[ais]

In [ ]:
np.array( [np.zeros((2,2)), np.zeros((2,2))] ).shape

In [ ]:
np.array([[4.4,5],[4,5]]).dtype == np.float

In [ ]:
tuple( np.array([[4,5],[4,5]]) )

In [ ]:
tuple( np.array(['fd','fds']) )